# A Toxic Comment Identifier Application - Main Notebook
### DSC 478 - Winter 2023
### Project Type: App Dev
### Team Members: Jeff Bocek, Xuyang Ji, Anna-Lisa Vu


### Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import cycle
import seaborn as sns
from collections import defaultdict

#from sklearn's...
from sklearn.feature_extraction.text import CountVectorizer #Convert a collection of text documents to a matrix of token counts.
from sklearn.feature_extraction.text import TfidfTransformer #Transform a count matrix to a normalized tf or tf-idf representation
from sklearn.feature_extraction.text import TfidfVectorizer #Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.model_selection import train_test_split, GridSearchCV #Cross validation gird search and training and testing chunks
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.naive_bayes import ComplementNB
from sklearn.decomposition import TruncatedSVD #Dimension Reduction (LSA)
from sklearn.preprocessing import Normalizer, LabelBinarizer 
from sklearn.ensemble import VotingClassifier #to create ensemble model
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.metrics import RocCurveDisplay, confusion_matrix, classification_report

## Exploratory Analysis (Xuyang)

In [2]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

## Pre-processing of Data (Xuyang)

In [3]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

### Loading Cleaned and Processed Data

Loading subset of training data with percentage of original instances:  
1% of non-toxic comments: 1433  
10% of toxic comments: 1370 and  
100% of sever toxic comments: 1595

Loading subset of testing data with percentage of original instances:  
1% of non-toxic comments: 577  
10% of toxic comments: 572 and   
100% of sever toxic comments: 367

In [4]:
file_object = open('clean_data1.p', 'rb')
clean_data = pickle.load(file_object)
file_object.close()
train = clean_data[0]
test = clean_data[1]

In [5]:
#balanced training data
train_df_subset1 = train['comment_text']
train_lab_subset1 = train['toxicity_level']

#testing data
test_df_subset1 = test['comment_text']
test_lab_subset1 = test['toxicity_level']

Loading **uneven** subset of training data with percentage of original instances:  
1% of non-toxic comments: 1433  
20% of toxic comments: 2740 and  
100% of sever toxic comments: 1595

In [6]:
file_object = open('clean_data_unbalanced.p', 'rb')
clean_data = pickle.load(file_object)
file_object.close()
train = clean_data[0]
test = clean_data[1]

In [7]:
#unbalanced training data
train_df_subsetU = train['comment_text']
train_lab_subsetU = train['toxicity_level']

#testing data
test_df_subsetU = test['comment_text']
test_lab_subsetU = test['toxicity_level']

## Data Transformation (Lisa)

In [8]:
#[Link to Toxic_App_Transformation notebook (if needed)]

### Term Frequencies

Simply put, the weights in the matrix represent the frequency of the term in a specific comment. The underlying concept is the higher the term frequency for a specific term in a comment, the more important it is for that comment. We use scikit-learn’s CountVectorizer(). Tuning occurred by adjusting the ‘max_df’ and ‘min_df’ which when building the vocabulary ignore terms that have a document frequency higher/lower respectively than the given threshold.

### TF*IDF

Term Frequency - Inverse Document Frequency assesses how important a term is within a comment relative to our collection of comments. It does this by vectorizing and scoring a term by multiplying the term’s Term Frequency (TF; number of times the term appears in the comment over the total number of terms in the comment)  by the Inverse Document Frequency (IDF; the log of; the total number of comments over the total number of comments which contain the specific term plus one). By using this formula we don’t place added importance on the super common words which overshadow less common words which can show more meaning to the comments. We used scikit-learn’s TfidfVectorizer() to transform our preprocessed collection of comments into a TF-IDF matrix. In aberration to the basic formula referenced above TfidfVectorizer() adds a “+1” to the numerator and denominator of the IDF score to prevent zero divisions when the term is not present to better represent the IDF scores for sparse matrices. We tuned this transformation by trying two methods of normalization; ‘l1’ and ‘l2’. ‘l1’  normalized by using the sum of the absolute values of the vector elements is 1 while ‘l2’ is the Euclidean norm and the sum of squares of vector elements is 1. For ‘l2’ the cosine similarity between two vectors is their dot product. As was the case with Term Frequency tuning, tuned by adjusting the ‘max_df’ and ‘min_df’ which when building the vocabulary ignore terms that have a document frequency higher/lower respectively than the given threshold. 

### Doc2Vec (Lisa)
Doc2Vec is a natural language processing technique to transform a set of documents into a list of vectors. It is closely related to the Word2Vec technique and is considered a more generalized form of it.  To best understand Doc2Vec, it is imperative to understad Word2Vec.  In Word2Vec, the algorithm looks at the context of each word and what other words surround it.  It assigns a numerical number to the word which represents the meaning of the word given its context.  For example. if Paris and France are in the same sentence, Word2Vec will create a number for each of these words which would represent that these 2 words are related.  This is exact same example is referenced in this article: https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e  Doc2Vec on the other hand generalized this technique.  Instead of having a numeric representation for the words, with Doc2Vec you can achieve a numeric representation for a document as a whole.

We thought it would be a good exercise to use this data transformation technique in our project.  Each comment can be like a document.  To create the Doc2Vec vectors, we used the doc2Vec library package from the gensim.models library.  We ran this against the train and test data and feed that into our models.

In the attached notebook are central methods we created so that we can get a doc2vec vector for the train and test data.  Using these methods each team member was able to feed a doc2vec transformed data set into the models.  Analysis and Evaluations of our the models performed with the data set is explained in the upcoming sections.

Link to Doc2Vec Notebook: [Toxic_App_Doc2Vec.ipynb](Toxic_App_Doc2Vec.ipynb)

In [10]:
from gensim.models import doc2vec
%run Toxic_App_Doc2Vec.ipynb
d2v_model = doc2vec.Doc2Vec(vector_size = 1000, min_count = 5, epochs = 40)

[{'dm': 0, 'vector_size': 500, 'window': 2, 'hs': 1, 'negative': 0}, {'dm': 0, 'vector_size': 500, 'window': 5, 'hs': 1, 'negative': 0}, {'dm': 0, 'vector_size': 1000, 'window': 2, 'hs': 1, 'negative': 0}, {'dm': 0, 'vector_size': 1000, 'window': 5, 'hs': 1, 'negative': 0}, {'dm': 1, 'vector_size': 500, 'window': 2, 'hs': 1, 'negative': 0}, {'dm': 1, 'vector_size': 500, 'window': 5, 'hs': 1, 'negative': 0}, {'dm': 1, 'vector_size': 1000, 'window': 2, 'hs': 1, 'negative': 0}, {'dm': 1, 'vector_size': 1000, 'window': 5, 'hs': 1, 'negative': 0}]


In [11]:
# Balanced training set
d2v_model = doc2vec.Doc2Vec(vector_size = 1000, min_count = 5, epochs = 40)
train_vectors1 = get_doc2vec_vectors(d2v_model, train_df_subset1)
tokenized_test_comments = tokenize_comments(test_df_subset1)
test_vectors1 = infer_vectors(d2v_model, tokenized_test_comments, "")

In [12]:
# Unbalanced training set
d2v_model = doc2vec.Doc2Vec(vector_size = 1000, min_count = 5, epochs = 40)
train_vectorsU = get_doc2vec_vectors(d2v_model, train_df_subsetU)
tokenized_test_comments = tokenize_comments(test_df_subsetU)
test_vectorsU = infer_vectors(d2v_model, tokenized_test_comments, "")

Data Dictionary

In [13]:
subsets = {
    # [X_train, y_train, X_test, y_test]
    'balanced': [train_df_subset1, train_lab_subset1, test_df_subset1, test_lab_subset1],
        # balanced training data with ratios 0.01, 0.10, 1.0
    'balanced_D2V': [train_vectors1, train_lab_subset1, test_vectors1, test_lab_subset1],
        # using Doc2Vec vectors
    'unbalanced': [train_df_subsetU, train_lab_subsetU, test_df_subsetU, test_lab_subsetU],
        # unbalanced training data with ratios 0.01, 0.20, 1.0
    'unbalanced_D2V': [train_vectorsU, train_lab_subsetU, test_vectorsU, test_lab_subsetU],
        # using Doc2Vec vectors
}

## Unsupervised Modeling for Exploratory Analysis
### Kmeans (Xuyang)

## Supervised Modeling & Hyperparameter Tuning

In [14]:
#import tuning and model evaluation functions
%run Toxic_App_Tuning_&_Evaluation_funcs.ipynb
pipe_param_dict = {}

### Gradiant Descent/Logistic Regression (Xuyang)

### Rocchio/Nearest Centroid (Lisa)

In the Rocchio/Nearest Centroid classifier algorithm, a prototype vector for each class is created from the training data.  The prototype vector is compared to each test instance and labels are assigned to test instances based on which prototype vector is closest to it.  The classifier can use different distance metrics to determine the nearest prototype vector.  Below, we use the nearest centroid classifier for both euclidean and cosine distance metrics.

In [18]:
# Rocchio Pipeline
pipe_Rocchio = Pipeline(
    [
        ('token_value', 'passthrough'),
        ('reduce_dim', 'passthrough'),
        ('clf', NearestCentroid())
    ]
)

# Rocchio pipeline with Doc2Vec
pipe_D2V_Rocchio = Pipeline(
    [
        ('reduce_dim', 'passthrough'),
        ('clf', NearestCentroid())
    ]
)

In [16]:
#Hyperparameter tunning

#Token 
min_DF = [1, 3, 10]
max_DF = [0.6, 0.8, 0.9]
#norm = ['l1','l2']

#Dimensionality Reduction
n_components = [115]

#Classifer 
metric = ['euclidean', 'cosine']
#shrink_threshold (for sinking centroids to remove fatures)

#Parameter dictionary for Rocchio Pipeline
Rocchio_params =[
    {
        'token_value': [CountVectorizer(), TfidfVectorizer()],
        'token_value__min_df': min_DF,
        'token_value__max_df': max_DF,
        'reduce_dim': ['passthrough'],
        'clf__metric': metric
    },
    {
        'token_value': [CountVectorizer(), TfidfVectorizer()],
        'token_value__min_df': min_DF,
        'token_value__max_df': max_DF,
        'reduce_dim': [TruncatedSVD()],
        'reduce_dim__n_components': n_components,
        'clf__metric': metric
    }
]

#Doc2Vec parameter dictionary for Rocchio pipeline
D2V_Rocchio_params =[
    {
        'reduce_dim': ['passthrough'],
        'clf__metric': metric
    },
    {
        'reduce_dim': [TruncatedSVD()],
        'reduce_dim__n_components': n_components,
        'clf__metric': metric
    }
]

In [17]:
#pipeline with parameter dictionary
pipe_param_dict['Rocchio_pipeline'] = [pipe_Rocchio, Rocchio_params]
pipe_param_dict['Doc2Vec_Rocchio_pipeline']= [pipe_D2V_Rocchio, D2V_Rocchio_params]

Tune Rocchio model without Doc2Vec via GridSearchCV()

In [20]:
Rocchio_best = run_on_subset(pipe_param_dict['Rocchio_pipeline'], subsets['balanced'])

/Users/lisasaurus01/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_nearest_centroid.py:164: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn(
/Users/lisasaurus01/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_nearest_centroid.py:164: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn(
/Users/lisasaurus01/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/lisasaurus01/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/lisasaurus01/opt/anaconda3/lib/python3.9/site-packages/sklear

KeyboardInterrupt: 

In [ ]:
#explanation of best parameters

Tune Rocchio model with Doc2Vec via GridSearchCV()

In [ ]:
Doc2Vec_Rocchio_best = run_on_subset(pipe_param_dict['Rocchio_pipeline'], subsets['balanced'])

In [ ]:
#explanation of best parameters

### Naive Bayes 

In [ ]:
# pipeline using term counts and complimentary Naive Bayes
pipe_CountVect_CompNB = Pipeline([
    ('count', CountVectorizer()),
    ('clf', ComplementNB())
    ])

# pipeline using Tfidf and complimentary Naive Bayes
pipe_TfidfVect_CompNB = Pipeline([
    ('vect', TfidfVectorizer()),
    ("clf", ComplementNB())
    ])

# pipeline using Doc2Vec and complimentary Naive Bayes is incompatible as passes negative values

In [ ]:
# Hyperparameter tunning

# parameters for CountVectorizer()
countvect_params = {
    'count__min_df': [1, 3, 10], 
    'count__max_df': [0.6, 0.8, 0.9]
    
}

# parameters for TfidfVectorizer()
vectorizer_params = {
    'vect__min_df': [1, 3, 10], 
    'vect__max_df': [0.6, 0.8, 0.9],
    'vect__norm': ['l1','l2'],
    'vect__binary': [False, True]
}

# parameters for Complimentary Naive Bayes()
compNB_params = {
    'clf__alpha' : [1, 10, 100]
}

In [ ]:
pipe_param_dict['CountVect_CompNB'] = [pipe_CountVect_CompNB, countvect_params]
pipe_param_dict['TfidfVect_CompNB'] = [pipe_TfidfVect_CompNB, vectorizer_params]

Tune Naive Bayes model with Term Frequencies via GridSearchCV()

In [ ]:
CV_NB_best = run_on_subset(pipe_param_dict['CountVect_CompNB'], subsets['balanced'])

In [ ]:
#explanation of best parameters

Tune Naive Bayes model with TFIDF via GridSearchCV()

In [ ]:
Tfidf_NB_best = run_on_subset(pipe_param_dict['TfidfVect_CompNB'], subsets['balanced'])

In [ ]:
#explanation of best parameters

### KNN

The K Nearest Neighbors model depends on a distance or similarity function to compare how far apart instances (data points) are away from each other. In classification, for a given number of points, the majority label of these “nearest neighbors” to our point in question is assigned to our point. To tune our KNN model we examined three parameters. The first was the number of neighbors (k). A larger k should reduce the effects of noise but will make the classification boundary fuzzier.

In [ ]:
# KNN Pipeline
pipe_KNN = Pipeline(
    [
        ('token_value', 'passthrough'),
        ('reduce_dim', 'passthrough'),
        ('clf', KNeighborsClassifier())
    ]
)

# Doc2Vec KNN Pipeline
pipe_D2V_KNN = Pipeline(
    [
        ('reduce_dim', 'passthrough'),
        ('clf', KNeighborsClassifier())
    ]
)

In [ ]:
#Hyperparameter tunning

#Token parameters
min_DF = [1, 3, 10]
max_DF = [0.6, 0.8, 0.9]
#norm = ['l1','l2']

#Dimensionality Reduction parameters
n_components = [115]

#Classifer parameters
n_neighbors = [5, 10]
weights = ['uniform', 'distance']
metric = ['minkowski', 'cosine']

#Parameter dictionary for KNN Pipeline
KNN_params =[
    {
        'token_value': [CountVectorizer(), TfidfVectorizer()],
        'token_value__min_df': min_DF,
        'token_value__max_df': max_DF,
        'reduce_dim': ['passthrough'],
        'clf__n_neighbors': n_neighbors,
        'clf__weights': weights,
        'clf__metric': metric
    },
    {
        'token_value': [CountVectorizer(), TfidfVectorizer()],
        'token_value__min_df': min_DF,
        'token_value__max_df': max_DF,
        'reduce_dim': [TruncatedSVD()],
        'reduce_dim__n_components': n_components,
        'clf__n_neighbors': n_neighbors,
        'clf__weights': weights,
        'clf__metric': metric
    }
]

#Doc2Vec parameter dictionary for KNN pipeline
D2V_KNN_params =[
    {
        'reduce_dim': ['passthrough'],
        'clf__n_neighbors': n_neighbors
    },
    {
        'reduce_dim': [TruncatedSVD()],
        'reduce_dim__n_components': n_components,
        'clf__n_neighbors': n_neighbors
    }
]

In [ ]:
#pipeline with parameter dictionary
pipe_param_dict['KNN_pipeline'] = [pipe_KNN, KNN_params]
pipe_param_dict['Doc2Vec_KNN_pipeline']= [pipe_D2V_KNN, D2V_KNN_params]

Tune KNN model without Doc2Vec via GridSearchCV()

In [ ]:
KNN_best = run_on_subset(pipe_param_dict['KNN_pipeline'], subsets['balanced'])

In [ ]:
#explanation of best parameters

Tune KNN model with Doc2Vec via GridSearchCV()

In [ ]:
Doc2Vec_KNN_best = run_on_subset(pipe_param_dict['Doc2Vec_KNN_pipeline'], subsets['balanced_D2V'])
# function from Toxic_App_Tuning_&_Evaluation_funcs.ipynb

In [ ]:
#explanation of best parameters

## Evaluation of Models - Model Comparisons

### Gradiant Descent/Logistic Regression (Xuyang)

### Rocchio

In [ ]:
analysis_no_pred_prob(subsets['balanced'], target_names, Rocchio_best)

In [ ]:
analysis_no_pred_prob(subsets['balanced'], target_names, Doc2Vec_Rocchio_best)

### KNN

In [ ]:
analysis(subsets['balanced'], target_names, KNN_best, y_score1)

In [ ]:
analysis(subsets['balanced'], target_names, Doc2Vec_KNN_best, y_score1)

### Naive Bayes

In [ ]:
analysis(subsets['balanced'], target_names, CV_NB_best, y_score1)

In [ ]:
analysis(subsets['balanced'], target_names, Tfidf_NB_best, y_score1)

### Model Evaluation Dataframe

In [ ]:
score_df = pd.DataFrame(scores).set_index("Classifier/Pipeline")
score_df = score_df.round(2)
score_df

### Top 3 Models with Unbalanced Dataset using Ensemble

In [ ]:
estimators = {
    ('NaiveBayes', Tfidf_NB_best),
    ('KNN', KNN_best),
    ('Rocchio', Rocchio_best)
}

#create the ensemble classifier
ensemble = VotingClassifier(estimators, voting = 'hard')

#fit the model to the training data
ensemble.fit(subsets['unbalanced'][0], subsets['unbalanced'][1])

#test the model
ensemble.score(subsets['unbalanced'][2], subsets['unbalanced'][3])

In [ ]:
analysis_no_pred_prob(subsets['unbalanced'], target_names, ensemble)